# Flow direction

## 1. Building the unstructured geometries

In [ ]:
import pycpt
import lavavu

import meshio
import numpy as np
import pygmsh as pg
import fillit as pitfill

from scipy.interpolate import RectBivariateSpline

from collections import OrderedDict

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable

import meshplex
from eSCAPE._fortran import defineGTIN

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

Some useful functions...

In [ ]:
def plotElevation( data, cmin, cmax, colormap):
    '''
    data: dataset to plot
    zmin,zmax: extent of the colormap
    colormap: to use    
    '''
    
    # Figure size is defined here
    fig = plt.figure(1, figsize=(6,6))
    
    ax = plt.gca()
    im = ax.imshow(data, interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)
    
def _distance(x, y, a, b, xo, yo):
        return (x-xo)**2/a**2 + (y-yo)**2/b**2

def _dome(c, zo, dist):
        return zo + c * np.sqrt(1. - dist )
    
def buildDome(img, a=None, b=None, c=None, base=None, xcenter=None, ycenter=None):
    """
    Build a simple dome surface topography.
    Parameters
    ----------
    variable : a,b,c
        Ellipsoid parameters.
    variable: base
        Base of the model in metres.
    variable: xcenter,ycenter
        X,Y coordinates for the centre of the dome.
    """

    nx = img.shape[0]
    ny = img.shape[1]
    
    newz = np.zeros((img.shape))
    for j in range(0,ny):
        for i in range(0,nx):
            dist = _distance(float(i),float(j),a,b,xcenter,ycenter)
            if dist < 1.:
                newz[i,j] = _dome(c, base, dist)

    return newz

def extractContours( X, Y, Z, cmin, cmax, colormap, ctrlist):
    '''
    coords: coordinate points (X,Y,X)
    cmin,cmax: extent of the colormap
    colormap: color scale to use
    ctrlist: list of contours to extract
    '''
    # Figure size is defined here
    fig = plt.figure(1, figsize=(6,6))
    ctrs = []
    for k in range(len(ctrlist)):
        ctrs.append(plt.contour(X, Y, 
                    np.flipud(Z), ctrlist[k]))
    ax = plt.gca()
    im = ax.imshow(Z, interpolation='nearest', cmap=colormap,
                     vmin=cmin, vmax=cmax,extent=[minX, maxX,minY, maxY])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(im,cax=cax)
    plt.tight_layout()

    plt.show()
    plt.close(fig)

    return ctrs

Set the initial surface

In [ ]:
spacing = 250.

img = -np.ones((500,500))

nx = img.shape[1]
ny = img.shape[0]

minX, maxX = 0, spacing*nx
minY, maxY = 0., spacing*ny

img3 = buildDome(img,a=200,b=200,c=1000,base=0.,xcenter=250,ycenter=250)

In [ ]:
xcoords = np.arange(minX, maxX, spacing)
ycoords = np.arange(minY, maxY, spacing)

X, Y = np.meshgrid(xcoords, ycoords)

coords = np.vstack([X.ravel(), Y.ravel()])

Build the **octopan** structured (based on quagmire's idea)

In [ ]:
x1 = np.arange(-6.25, 6.25, 0.025)
y1 = np.arange(-6.25, 6.25, 0.025)

X1, Y1 = np.meshgrid(x1, y1)

coords1 = np.vstack([X1.ravel(), Y1.ravel()])

xx = coords1[0,:]
yy = coords1[1,:]

radius  = np.sqrt((xx**2 + yy**2))
theta   = np.arctan2(yy,xx)

height  = np.exp(-0.025*(xx**2 + yy**2)**2) + 0.25 * (0.2*radius)**4  * np.cos(10.0*theta)**2 
height  += 0.5 * (1.0-0.2*radius)

height = np.multiply(height,666.)+155.

In [ ]:
ctrList = [0.]
topocmap = pycpt.load.cmap_from_cptcity_url('td/DEM_screen.cpt')

# Now we extract the contours lines using the previous function
if sorted(ctrList) == ctrList:
    ctrs = extractContours(X, Y, img3, -100, 1000, topocmap, ctrList)
else:
    print("ERROR:")
    print("The list of contour positions needs to be specify in ascending order!")
    print("Redefine the ctrList variable")

Extract the contour and 'cut' the domain based on this contour...

In [ ]:
def distancePts(p1,p2):
    '''
    Compute the euclidian distance between 2 points (p1, p2)
    
    '''
    return (p1[1]-p2[1])**2+(p1[0]-p2[0])**2 


def getLongestContoursXY(ctrs, lcar):
    '''
    1- Extract from the list of contour points the longest path
    2- Using a characteristic length (lcar) resample the path 
    3- Return the XY coordinates of the longest paths
    '''

    ctrPoints = []
    # Loop through the contour lines 
    for ct in range(len(ctrs)):
        cpath = []
        k = 0
        maxpts = 0
        pathID = 0
        
        # For each contour extract the longest path
        for collection in ctrs[ct].collections:
            for path in collection.get_paths():
                if len(path)>4:
                    cpath.append(np.asarray(path.to_polygons()[0]))
                    # Storing longest path
                    if cpath[-1].shape[0] > maxpts:
                        maxpts =  cpath[-1].shape[0]
                        pathID = k
                    k += 1

        # Find longest path XY coordinates 
        Cpts = cpath[pathID]
        x = Cpts[:,0]
        y = Cpts[:,1]
        tmp = OrderedDict()
        for pt in zip(x,y):
            tmp.setdefault(pt[:1], pt)   
        ctrPts = np.asarray(tmp.values())  

        # Resample the path to the requested characteristic length
        ki = 0
        tmpPts = []
        cumdist = 0.
        tmpPts.append(ctrPts[0,:])
        for k in range(1,ctrPts.shape[0]):
            cumdist = distancePts(ctrPts[ki,:2], ctrPts[k,:2])
            if(cumdist >= lcar):
                tmpPts.append(ctrPts[k,:])
                ki = k
        ctrPoints.append(np.asarray(tmpPts))

    return ctrPoints



In [ ]:
cpath = []
for collection in ctrs[0].collections:
    for path in collection.get_paths():
        if len(path)>4:
            cpath.append(np.asarray(path.to_polygons()[0]))
            
ctrPoints = getLongestContoursXY(ctrs, 2.)

### Using pyGmsh library

Using `pyGmsh` library, which is a Python interface to `Gmsh`: a powerful mesh generation tool:

- https://github.com/nschloe/pygmsh

The resolution of the mesh is defined based on the characteristic length parameter `lcar`.

There are several different ways of defining the mesh, here we use an approach similar to what we've done previously with `stripy` and use the border coordinates to create our delaunay triangulation.

#### Bounding box

We first initialise a pyGmsh instance and define the characteristc length `lcar`:

In [ ]:
geom = pg.built_in.Geometry()

# Gmsh options
gmsh_out = True
gmsh_args = ['-clmin', '400', '-smooth', '10']

In [ ]:
def GmeshPolyline(GmshGeo, ctrPts, lcar):
    '''
    Define a polyline for Gmsh geometry (GmshGeo) based on a list of coonected 
    points (contour line ctrPts) and a characteristic length (lcar)
    '''
    Points_done=dict()
    Line_loops=dict()
    Line_done=dict()

    # 1- Contour coordinates definition
    for i in range(ctrPts.shape[0]):
        Points_done[i]=GmshGeo.add_point([ctrPts[i][0], ctrPts[i][1], 0.], lcar)

    # 2- Lines between points definition
    lineLoop=[]
    for i in range(ctrPts.shape[0]):
        if i < ctrPts.shape[0]-1:
            Line_done[i]=GmshGeo.add_line(Points_done[i],Points_done[i+1])
        else:
            Line_done[i]=GmshGeo.add_line(Points_done[i],Points_done[0])
        lineLoop.append(Line_done[i])

    return GmshGeo.add_line_loop(lineLoop)

In [ ]:
# External contour Gmsh characteristic length
lcar1 = 400.

# Create polyline for contour line at -1000 m depth
polyline1 = GmeshPolyline(geom, cpath[0], lcar1)

In [ ]:
# Adding refined surface in the region within the second polyline
geom.add_plane_surface(polyline1) 

# Generate Gmsh triangulation
mesh = pg.generate_mesh(geom, dim = 2, 
                                       verbose = gmsh_out,
                                       extra_gmsh_arguments=gmsh_args)

In [ ]:
pts = mesh.points
cells = mesh.cells

In [ ]:
print("Unstructured Grid:\n")

print("Number of points         nbpt: {}".format(pts.shape[0]))
print("Number of faces       nbcells: {}".format(cells['triangle'].shape[0]))

In [ ]:
interpolateFct = RectBivariateSpline(xcoords,ycoords,np.flipud(height.reshape((500,500))).T)

In [ ]:
# Interpolation of the elevation on the triangulation
evalZ = interpolateFct.ev(pts[:,0], pts[:,1])
evalZ[evalZ < min(ctrList)] = min(ctrList) 

# Creation of the X,Y,Z coordinates of the unstructured grid
verts = np.insert(pts[:,:2], 2, values=evalZ, axis=1)

In [ ]:
lv = lavavu.Viewer(border=True, background="#FFFFFF", resolution=[700,700], near=-10.0)

# Core 
lvTriG = lv.triangles("DelGmsh",  wireframe=False, colour="#161616", opacity=1.0)
lvTriG.vertices(verts)
lvTriG.indices(cells['triangle'])
lvTriG.values(evalZ)
lvTriG.colourmap("dem1", range=[0.,1200.])
lv.translation(0.0, 0.0, -251736.313)
lv.rotation(-63.015, 1.826, -0.655)
lv.scale('z', 75)

lv.control.Checkbox(property='axis')
lvTriG.control.Checkbox(property='wireframe', label="wireframe")
lv.control.Panel()
lv.control.ObjectList()
lvTriG.control.Range(command='scale z', range=(1,201), step=10., value=10)
lv.control.show()

In [ ]:
mesh = meshio.Mesh(pts, cells, {'Z':evalZ})
meshio.write("octopan.vtk", mesh)

## 2. Running the model 

All the input parameters are specified in the `input.yml` file.

### Input file

Input files for **eSCAPE** are based on [YAML](https://circleci.com/blog/what-is-yaml-a-beginner-s-guide/) syntax.

```YAML
name: Testing different flow direction

domain:
    filename: ['octopan.vtk','Z']
    flowdir: 12
    bc: 'fixed'

time:
    start: 0
    end: 100000.
    tout: 5000.
    dt: 1000.

sea:
    position: -100.

climate:
    - start: 0.
      uniform: 1
     
sp_br:
    Kbr: 2.e-5

sp_dep:
    Ff: 0.3

diffusion:
    hillslopeK: 1.e-2
    sedimentK: 200. 

output:
    dir: 'octopanMFD'
    makedir: False

```

#### Parameters 

+ `domain`: definition of the unstructured grid containing the vtk grid `filename` and the associated field (here called `Z`) as well as the flow direction method to be used `flowdir` that takes an integer value between 1 (for SFD) and 12 (for Dinf) and the boundary conditions (`bc`: 'flat', 'fixed' or 'slope')

+ `time`: the simulation time parameters defined by `start`, `end`, `tout` (the output interval) and `dt` (the internal time-step).

Follows the optional forcing conditions:

+ `sea`: the sea-level declaration with the relative sea-level `position` (m) and the sea-level `curve` which is a file containing 2 columns (time and sea-level position).

+ `climatic` & `tectonic` have the same structure with a sequence of events defined by a starting time (`start`) and either a constant value (`uniform`) or a `map`.

Then the parameters for the surface processes to simulate:

+ `sp_br`: for the _stream power law_ with a unique parameter `Kbr` representing the The erodibility coefficient which is scale-dependent and its value depend on lithology and mean precipitation rate, channel width, flood frequency, channel hydraulics. It is worth noting that the coefficient _m_ and _n_ are fixed in this version and take the value 0.5 & 1 respectively. In this example we consider that all eroded sediments  are transported as fine suspension `Ff`=1 and as such will never be redeposited.

+ `diffusion`: hillslope, stream and marine diffusion coefficients. `hillslopeK` sets the _simple creep_ transport law which states that transport rate depends linearly on topographic gradient. The marine sediment are transported based on a diffusion coefficient `sedimentK`. 

Finally, you will need to specify the output folder:

+ `output`: with `dir` the directory name and the option `makedir` that gives the possible to delete any existing output folder with the same name (if set to False) or to create a new folder with the give `dir` name plus a number at the end (e.g. outputDir_1 if set to True)

### Using Jupyter notebook environment

For small models it is possible to use the notebook environment directly and run the following set of commands:

```python
import eSCAPE

# Reading input file
model = eSCAPE.LandscapeEvolutionModel(input_globe.yml)

# Running model
model.runProcesses()

# Running model
model.destroy()
```

### Using Python file

Here we will use a Python script called `run_escape.py` located in the same folder as your input file. 

```python
import argparse
import eSCAPE as sim

# Parsing command line arguments
parser = argparse.ArgumentParser(description='This is a simple entry to run eSCAPE model.',add_help=True)
parser.add_argument('-i','--input', help='Input file name (YAML file)',required=True)
parser.add_argument('-v','--verbose',help='True/false option for verbose', required=False,action="store_true",default=False)
parser.add_argument('-l','--log',help='True/false option for PETSC log', required=False,action="store_true",default=False)

args = parser.parse_args()
if args.verbose:
  print("Input file: {}".format(args.input))
  print(" Verbose is on? {}".format(args.verbose))
  print(" PETSC log is on? {}".format(args.log))

# Reading input file
model = sim.LandscapeEvolutionModel(args.input,args.verbose,args.log)

# Running model
model.runProcesses()

# Cleaning model
model.destroy()
```

This script is basically equivalent to what you will have done in the Jupyter environment but can also be ran on multiple processors using the `mpirun` command as shown below:

In [ ]:
!mpirun -np 4 python run_escape.py -i input.yml